# Projet

## Importing Pyspark libs & reading files

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName('abc').getOrCreate()

df1=spark.read.format("csv").option("header","true").option("inferSchema","true").load('ApplePrices.csv')

df2=spark.read.format("csv").option("header","true").option("inferSchema","true").load('CurrencyConversion.csv')

df1.show()

df2.show()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/14 08:41:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+--------------------+-------+---------+--------+
|          Model_name|  Price|  Country|Currency|
+--------------------+-------+---------+--------+
|        24-inch iMac|1919.01|Australia|     AUD|
|         AirPods Max| 908.47|Australia|     AUD|
|         AirPods Pro|  403.2|Australia|     AUD|
|AirPods(2nd gener...| 221.31|Australia|     AUD|
|AirPods(3rd gener...| 281.94|Australia|     AUD|
|Apple Pencil (2nd...|  201.1|Australia|     AUD|
|         Apple TV 4K| 251.62|Australia|     AUD|
|      Apple Watch SE| 433.52|Australia|     AUD|
|Apple Watch Series 3| 302.15|Australia|     AUD|
|         MacBook Air| 1514.8|Australia|     AUD|
|         Magic Mouse| 110.15|Australia|     AUD|
|          Sport Band|  69.73|Australia|     AUD|
|                iPad| 504.26|Australia|     AUD|
|            iPad Pro|1211.64|Australia|     AUD|
|           iPhone 12|1009.53|Australia|     AUD|
|           iPhone 13|1211.64|Australia|     AUD|
|           iPhone SE| 686.16|Australia|     AUD|


## Join the dataframes & conversion
Firt we need to create a join expression on our two dataframe on the sole link they share: the currency/ISO_4217 designation of the currency.
That will give us a new dataframe with line by line comparison between the original price in each country and the dollar to currency ratio.

In [2]:
join_express = df1["Currency"] == df2["ISO_4217"]
joined = df1.join(df2, join_express, "inner")

We will then create a new column containing the converted dollar price for each product.

In [3]:
joined_and_converted_prices = joined.withColumn("Dollar Prices", round(expr("Price / Dollar_To_Curr_Ratio"), 2))
joined_and_converted_prices.show()

+--------------------+-------+---------+--------+--------+--------------------+-------------+
|          Model_name|  Price|  Country|Currency|ISO_4217|Dollar_To_Curr_Ratio|Dollar Prices|
+--------------------+-------+---------+--------+--------+--------------------+-------------+
|        24-inch iMac|1919.01|Australia|     AUD|     AUD|                 1.4|      1370.72|
|         AirPods Max| 908.47|Australia|     AUD|     AUD|                 1.4|       648.91|
|         AirPods Pro|  403.2|Australia|     AUD|     AUD|                 1.4|        288.0|
|AirPods(2nd gener...| 221.31|Australia|     AUD|     AUD|                 1.4|       158.08|
|AirPods(3rd gener...| 281.94|Australia|     AUD|     AUD|                 1.4|       201.39|
|Apple Pencil (2nd...|  201.1|Australia|     AUD|     AUD|                 1.4|       143.64|
|         Apple TV 4K| 251.62|Australia|     AUD|     AUD|                 1.4|       179.73|
|      Apple Watch SE| 433.52|Australia|     AUD|     AUD|  

A little cleaning of the duplicates/useless columns later.

In [4]:
joined_and_converted_prices = joined_and_converted_prices.drop("ISO_4217").drop("Dollar_To_Curr_Ratio")
joined_and_converted_prices.show(1000)

+--------------------+---------+--------------+--------+-------------+
|          Model_name|    Price|       Country|Currency|Dollar Prices|
+--------------------+---------+--------------+--------+-------------+
|        24-inch iMac|  1919.01|     Australia|     AUD|      1370.72|
|         AirPods Max|   908.47|     Australia|     AUD|       648.91|
|         AirPods Pro|    403.2|     Australia|     AUD|        288.0|
|AirPods(2nd gener...|   221.31|     Australia|     AUD|       158.08|
|AirPods(3rd gener...|   281.94|     Australia|     AUD|       201.39|
|Apple Pencil (2nd...|    201.1|     Australia|     AUD|       143.64|
|         Apple TV 4K|   251.62|     Australia|     AUD|       179.73|
|      Apple Watch SE|   433.52|     Australia|     AUD|       309.66|
|Apple Watch Series 3|   302.15|     Australia|     AUD|       215.82|
|         MacBook Air|   1514.8|     Australia|     AUD|       1082.0|
|         Magic Mouse|   110.15|     Australia|     AUD|        78.68|
|     

## That average question
For comprehension and handling I decided to clean (drop/rename) columns of my dataframe.
The idea will be create a USA products only dataframe and then to join on the model name column to get line by line a comparison between a product in a country and its price in the United States.

In [5]:
cleaned_version = joined_and_converted_prices\
.drop("Price")\
.drop("Currency")
cleaned_version.show()

+--------------------+---------+-------------+
|          Model_name|  Country|Dollar Prices|
+--------------------+---------+-------------+
|        24-inch iMac|Australia|      1370.72|
|         AirPods Max|Australia|       648.91|
|         AirPods Pro|Australia|        288.0|
|AirPods(2nd gener...|Australia|       158.08|
|AirPods(3rd gener...|Australia|       201.39|
|Apple Pencil (2nd...|Australia|       143.64|
|         Apple TV 4K|Australia|       179.73|
|      Apple Watch SE|Australia|       309.66|
|Apple Watch Series 3|Australia|       215.82|
|         MacBook Air|Australia|       1082.0|
|         Magic Mouse|Australia|        78.68|
|          Sport Band|Australia|        49.81|
|                iPad|Australia|       360.19|
|            iPad Pro|Australia|       865.46|
|           iPhone 12|Australia|       721.09|
|           iPhone 13|Australia|       865.46|
|           iPhone SE|Australia|       490.11|
|        24-inch iMac|   Canada|      1258.24|
|         Air

In [6]:
usa = joined_and_converted_prices.filter("Country = 'United States'")\
.drop("Country")\
.drop("Currency")\
.drop("Price")\
.withColumnRenamed("Dollar Prices", "USA_Prices")\
.withColumnRenamed("Model_name", "Model")
usa.show()

+--------------------+----------+
|               Model|USA_Prices|
+--------------------+----------+
|        24-inch iMac|    1299.0|
|         AirPods Max|     549.0|
|         AirPods Pro|     249.0|
|AirPods(2nd gener...|     129.0|
|AirPods(3rd gener...|     179.0|
|Apple Pencil (2nd...|     129.0|
|         Apple TV 4K|     179.0|
|      Apple Watch SE|     279.0|
|Apple Watch Series 3|     199.0|
|         MacBook Air|     999.0|
|         Magic Mouse|      79.0|
|          Sport Band|      49.0|
|                iPad|     329.0|
|            iPad Pro|     799.0|
|           iPhone 12|     599.0|
|           iPhone 13|     699.0|
|           iPhone SE|     399.0|
+--------------------+----------+



In [7]:
join_express = cleaned_version["Model_name"] == usa["Model"]
joined = cleaned_version.join(usa, join_express, "inner").drop("Model")
joined.show()

+--------------------+---------+-------------+----------+
|          Model_name|  Country|Dollar Prices|USA_Prices|
+--------------------+---------+-------------+----------+
|        24-inch iMac|Australia|      1370.72|    1299.0|
|         AirPods Max|Australia|       648.91|     549.0|
|         AirPods Pro|Australia|        288.0|     249.0|
|AirPods(2nd gener...|Australia|       158.08|     129.0|
|AirPods(3rd gener...|Australia|       201.39|     179.0|
|Apple Pencil (2nd...|Australia|       143.64|     129.0|
|         Apple TV 4K|Australia|       179.73|     179.0|
|      Apple Watch SE|Australia|       309.66|     279.0|
|Apple Watch Series 3|Australia|       215.82|     199.0|
|         MacBook Air|Australia|       1082.0|     999.0|
|         Magic Mouse|Australia|        78.68|      79.0|
|          Sport Band|Australia|        49.81|      49.0|
|                iPad|Australia|       360.19|     329.0|
|            iPad Pro|Australia|       865.46|     799.0|
|           iP

In [8]:
avg_prices = joined.groupBy("Country")\
.avg("Dollar Prices", "USA_PRICES")\
.sort("Country")\
.select("Country", round("`avg(Dollar Prices)`", 2), round("`avg(USA_PRICES)`", 2)) \
.withColumnRenamed("round(avg(Dollar Prices), 2)", "Moyenne")\
.withColumnRenamed("round(avg(USA_PRICES), 2)", "Moyenne aux USA")
avg_prices.show(100)

+--------------+-------+---------------+
|       Country|Moyenne|Moyenne aux USA|
+--------------+-------+---------------+
|     Australia| 472.28|         420.18|
|       Austria| 533.52|         420.18|
|        Canada|  432.0|         420.18|
|Czech Republic| 546.03|         412.13|
|       Denmark| 547.29|         412.13|
|       Finland| 538.72|         412.13|
|        France|  537.5|         420.18|
|       Germany| 532.44|         420.18|
|       Hungary| 530.84|         412.13|
|         India|  547.0|         420.18|
|       Ireland| 545.47|         420.18|
|         Italy|  523.9|         402.75|
|    Luxembourg| 509.31|         412.13|
|        Mexico| 507.62|         402.75|
|   Netherlands| 526.95|         412.13|
|        Norway| 548.22|         412.13|
|   Philippines|  477.6|         412.13|
|        Poland| 523.36|         412.13|
|      Portugal| 538.72|         412.13|
|        Russia| 504.75|          393.0|
|         Spain| 511.91|         402.75|
|        Sweden|

In [9]:
moyennePrix = avg_prices.withColumn("Ecart entre la moyenne et celle des USA(%)", round(expr(f"(abs(Moyenne) - `Moyenne aux USA`)/ `Moyenne aux USA` * 100"), 2))\
.sort(desc("Ecart entre la moyenne et celle des USA(%)"))
moyennePrix.show(100)

+--------------+-------+---------------+------------------------------------------+
|       Country|Moyenne|Moyenne aux USA|Ecart entre la moyenne et celle des USA(%)|
+--------------+-------+---------------+------------------------------------------+
|        Sweden|  551.4|         412.13|                                     33.79|
|        Norway| 548.22|         412.13|                                     33.02|
|       Denmark| 547.29|         412.13|                                      32.8|
|Czech Republic| 546.03|         412.13|                                     32.49|
|       Finland| 538.72|         412.13|                                     30.72|
|      Portugal| 538.72|         412.13|                                     30.72|
|         India|  547.0|         420.18|                                     30.18|
|         Italy|  523.9|         402.75|                                     30.08|
|       Ireland| 545.47|         420.18|                                    

In [10]:
moyennePrix.repartition(1).write.mode("overwrite").format("csv").option("header", "true").save("moyennePrix")

## Sum

In [11]:
total_count = joined_and_converted_prices.groupBy("Country")\
.agg({"Dollar Prices": "sum"})\
.select("Country", round("sum(Dollar Prices)", 2))\
.withColumnRenamed("round(sum(Dollar Prices), 2)", "Somme des produits")\
.sort(desc("sum(Dollar Prices)"))\

total_count.show(1000)
total_count.repartition(1).write.mode("overwrite").format("csv").option("header", "true").save("coutTotal")

+--------------+------------------+
|       Country|Somme des produits|
+--------------+------------------+
|         Italy|           9724.55|
|         India|           9512.19|
|         Spain|           9498.79|
|        Mexico|           9390.35|
|       Ireland|           9273.05|
|       Austria|           9249.34|
|        France|           9137.58|
|        Russia|           9082.05|
|       Germany|           9051.47|
|        Sweden|           9008.66|
|        Norway|           8962.36|
|       Denmark|           8953.71|
|Czech Republic|           8927.81|
|      Portugal|           8798.95|
|       Finland|           8798.95|
|       Hungary|           8679.36|
|   Netherlands|           8610.66|
|        Poland|           8558.95|
|    Luxembourg|           8322.53|
|     Australia|           8028.75|
|   Philippines|           7816.77|
|        Canada|           7344.05|
| United States|            7143.0|
|      Thailand|            7096.8|
+--------------+------------

## Liste des produits

In [12]:
product_list = df1.dropDuplicates(["Model_name"]).sort("Model_name")
product_list.show(1000)
product_list.repartition(1).write.mode("overwrite").format("csv").option("header", "true").save("listeProduit")

+--------------------+-------+---------+--------+
|          Model_name|  Price|  Country|Currency|
+--------------------+-------+---------+--------+
|        24-inch iMac|1919.01|Australia|     AUD|
|         AirPods Max| 908.47|Australia|     AUD|
|         AirPods Pro|  403.2|Australia|     AUD|
|AirPods(2nd gener...| 221.31|Australia|     AUD|
|AirPods(3rd gener...| 281.94|Australia|     AUD|
|Apple Pencil (2nd...|  201.1|Australia|     AUD|
|         Apple TV 4K| 251.62|Australia|     AUD|
|         Apple TV HD| 1299.0|  Denmark|     DKK|
|      Apple Watch SE| 433.52|Australia|     AUD|
|Apple Watch Series 3| 302.15|Australia|     AUD|
|         MacBook Air| 1514.8|Australia|     AUD|
|         Magic Mouse| 110.15|Australia|     AUD|
|          Sport Band|  69.73|Australia|     AUD|
|                iPad| 504.26|Australia|     AUD|
|            iPad Pro|1211.64|Australia|     AUD|
|           iPhone 12|1009.53|Australia|     AUD|
|           iPhone 13|1211.64|Australia|     AUD|


## Pays le moins cher pour acheter les AirPods Pro

In [13]:
airpodsPro = joined_and_converted_prices.filter("Model_name == 'AirPods Pro'").sort(asc("Dollar Prices")).limit(1)
airpodsPro.show()

+-----------+-----+-------------+--------+-------------+
| Model_name|Price|      Country|Currency|Dollar Prices|
+-----------+-----+-------------+--------+-------------+
|AirPods Pro|249.0|United States|     USD|        249.0|
+-----------+-----+-------------+--------+-------------+



In [14]:
airpodsPro.repartition(1).write.mode("overwrite").format("csv").option("header", "true").save("airpodsPro")